In [1]:
import os
import torch
import requests
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
from transformers import WhisperTimeStampLogitsProcessor

In [2]:
AUDIO_PATH = 'downloaded_audio.mp3'
AUDIO_URL = "https://storage.googleapis.com/radiofilez/english/saturday/kfm_93_3-2022-11-26_T11.00.01.mp3"

if not os.path.exists(AUDIO_PATH):
    response = requests.get(AUDIO_URL, allow_redirects=True)

    if response.status_code == 200:
        with open(AUDIO_PATH, 'wb') as f:
            f.write(response.content)
        print(f"Audio file downloaded successfully: {AUDIO_PATH}")
    else:
        print(f"Failed to download audio file: {response.status_code}")

Audio file downloaded successfully: downloaded_audio.mp3


In [3]:
device           = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
model_id      = "openai/whisper-base.en"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype        = torch_dtype,
    use_safetensors = True
)
model.to(device)

processor = AutoProcessor.from_pretrained(
    model_id,

)

config.json:   0%|          | 0.00/1.94k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.53k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/805 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.41M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.83k [00:00<?, ?B/s]

In [4]:
pipe = pipeline(
    task = "automatic-speech-recognition",
    model = model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    return_timestamps=True,
    torch_dtype=torch_dtype,
    device=device,
)
result = pipe(
    AUDIO_PATH,
)

Whisper did not predict an ending timestamp, which can happen if audio is cut off in the middle of a word. Also make sure WhisperTimeStampLogitsProcessor was used during generation.


In [5]:
result.keys()

dict_keys(['text', 'chunks'])

In [6]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')
sentiment_classifier = pipeline(
    task = "sentiment-analysis"
)

for chunk in result['chunks']:
    if len(chunk['text']) <= 1: continue
    sentiment = sentiment_classifier(chunk['text'])[0]
    chunk['sentiment_label'] = sentiment['label']
    chunk['sentiment_score'] = sentiment['score']

    print(chunk)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

{'timestamp': (0.0, 8.96), 'text': ' Did you know that teenage pregnancies contribute 20% of infant deaths and 28% of maternal deaths?', 'sentiment_label': 'NEGATIVE', 'sentiment_score': 0.9952617883682251}
{'timestamp': (8.96, 15.84), 'text': ' Join the conversation to speak out and help in ending child marriages and teenage pregnancies in Uganda.', 'sentiment_label': 'POSITIVE', 'sentiment_score': 0.9989373087882996}
{'timestamp': (15.84, 21.24), 'text': ' This message is brought to you by Action Aid in partnership with UNFBA UNICEF,', 'sentiment_label': 'POSITIVE', 'sentiment_score': 0.9970641732215881}
{'timestamp': (21.24, 37.4), 'text': ' Ministry of Gender, Labour and Social Development. by nature. Prezori milo water is your trusted source to help you stay healthy, hydrated and refreshed.', 'sentiment_label': 'POSITIVE', 'sentiment_score': 0.9991033673286438}
{'timestamp': (37.4, 42.24), 'text': " Prezori is from a mirror-rich underground source that delivers nature's goodness i

In [7]:
import json
import pandas as pd
with open ('output.json','w') as fp:
    json.dump(fp=fp,obj=result['chunks'],indent=4)

In [8]:
import json
with open ('output.json','r') as fp:
    result_chunks = json.load(fp=fp)

In [9]:
from sqlalchemy import Column, Integer, Unicode, UnicodeText, String
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.sql.schema import  Sequence
from sqlalchemy.orm import declarative_base


engine = create_engine('sqlite:///results.db', echo=True)
Base = declarative_base()

class Transcription(Base):
    __tablename__ = 'transcriptions'
    id = Column(Integer,Sequence('my_seq', start = 1), primary_key=True, )
    timestamp = Column('timestamp',String())
    text = Column('text',String())
    sentiment_label = Column('sentiment_label',String())
    sentiment_score = Column('sentiment_score',String())

    def __init__(self, timestamp, text,sentiment_label,sentiment_score):
        self.timestamp = timestamp
        self.text = text
        self.sentiment_label = sentiment_label
        self.sentiment_score = sentiment_score

Base.metadata.create_all(bind=engine)

Session = sessionmaker(bind=engine)
mySession = Session()

2024-06-21 08:08:42,193 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-06-21 08:08:42,199 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("transcriptions")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("transcriptions")


2024-06-21 08:08:42,203 INFO sqlalchemy.engine.Engine [raw sql] ()


...

INFO:sqlalchemy.engine.Engine:
CREATE TABLE transcriptions (
	id INTEGER NOT NULL, 
	timestamp VARCHAR, 
	text VARCHAR, 
	sentiment_label VARCHAR, 
	sentiment_score VARCHAR, 
	PRIMARY KEY (id)
)




2024-06-21 08:08:42,215 INFO sqlalchemy.engine.Engine [no key 0.00244s] ()


INFO:sqlalchemy.engine.Engine:[no key 0.00244s] ()


2024-06-21 08:08:42,226 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [10]:
for transcription in result_chunks:
    timestamp       = str(transcription['timestamp'])
    text            = str(transcription['text'])
    sentiment_label = str(transcription['sentiment_label'])
    sentiment_score = str(transcription['sentiment_score'])
    transcript = Transcription(timestamp,text,sentiment_label,sentiment_score)
    mySession.add(transcript)
    mySession.commit()

2024-06-21 08:08:42,237 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2024-06-21 08:08:42,244 INFO sqlalchemy.engine.Engine INSERT INTO transcriptions (timestamp, text, sentiment_label, sentiment_score) VALUES (?, ?, ?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO transcriptions (timestamp, text, sentiment_label, sentiment_score) VALUES (?, ?, ?, ?)


2024-06-21 08:08:42,246 INFO sqlalchemy.engine.Engine [generated in 0.00204s] ('[0.0, 8.96]', ' Did you know that teenage pregnancies contribute 20% of infant deaths and 28% of maternal deaths?', 'NEGATIVE', '0.9952617883682251')


...

INFO:sqlalchemy.engine.Engine:INSERT INTO transcriptions (timestamp, text, sentiment_label, sentiment_score) VALUES (?, ?, ?, ?)


2024-06-21 08:08:58,651 INFO sqlalchemy.engine.Engine [cached since 16.41s ago] ('[3580.08, None]', ' league. While you can mention their ability and all, some', 'POSITIVE', '0.9950856566429138')


INFO:sqlalchemy.engine.Engine:[cached since 16.41s ago] ('[3580.08, None]', ' league. While you can mention their ability and all, some', 'POSITIVE', '0.9950856566429138')


2024-06-21 08:08:58,656 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT
